# To start a PyCBC run:

Everything you need to start your runs is in this directory **which you need to copy** to where you'd like to run from:

In [ ]:
cp /home/hannah.griggs/nu/pynu_tests/skyloc/run_testing/events_test/o3events/runmodule path/to/rundirectory

Once you copy that directory, you will need to adjust some of the files to point to your directories/namespace. To see all of the places in here that point to MY directory, run 

In [ ]:
grep -i "hannah.griggs" *

This should print all of the files that contain "hannah.griggs" and the line it's in. 

**You need to change the kinit and INI_LOC lines in run.sh (and runcache.sh)**. 

If you are running a plain HL job, no need to change statistic-files in analysis_dtdp_noinj.ini. If you are running a job with a custom dtdp pdf, that line needs to change with the location of the pdf.

**Launch the necessary environment (pynumods)**. Note that the environment command lives in "environmentnew.sh". This is just to make your life easier so you don't have to find the full path of the environment every time you want to launch it. 

You must deactivate the (igwn) environment first, then source the modified pynu environment like this with the "." followed by a space followed by the bash script.

In [ ]:
conda deactivate # Get out of the (igwn) environment. Make sure to have no active environment at all before sourcing.
. environmentnew.sh # Activate the pynu environment. The ". " command tells your shell to accept the change for the remainder of the shell session.

## Part 1: Running an analysis:

If running with reused data, see Part 2. If running with new data, START here.

In the runmodule directory are ini files for the pycbc workflow. To note here are:
1. analysis_dtdp.ini (contains adjusted dtdp pdf input line, you'll want to edit this with your custum pdf)
2. analysis_hl.ini (contains original dtdp PDF. Use this for INITIAL runs for NEW analyses.)
3. inspiral.ini (contains injection-file, which controls which injection is used for this analysis)
5. run.sh (the run script)
6. runcache.sh (the run script for dtdp runs, has cache file option and the dtdp analysis file)

### **run.sh** will need to be edited as:

In [ ]:
WORKFLOW_NAME=mmatest ## you can keep this the same
CONFIG_TAG=v2.3.2.3  ## keep this
GITLAB_URL="https://git.ligo.org/pycbc/offline-analysis/-/raw/${CONFIG_TAG}/production/o4/broad/config"
ID=injection ## Change this to indicate if it's a plain injection or a dtdp run
RUNID=_150 ## Change to reflect the injected distance that goes with the dtdphase PDF you made AND any other info you need to keep track of

In [ ]:
INI_LOC="/home/hannah.griggs/nu/pynu_tests/skyloc/run_testing/events_test/o3events/" # Change to your runmodule location

In [ ]:
kinit hannah.griggs ## Change to your ligo name

### Point your analysis to the correct injection file. 

Injection files live in my directory:

In [ ]:
/home/hannah.griggs/nu/banks/gen_plot_injections/autoinjections/pycbcinjs/

The injections we're using have the following characteristics:

In [ ]:
tc = 1239880055.862449 ## Central time that matches set of dtdp pdfs
mass1 = 1.8 ## BNS
mass2 = 1.5
ra = 158.9333 ## Static sky location that matches set of dtdp pdfs
dec = 46.4552
inclination = 45.0
coa_phase = 0.0
polarization = 0.0
f_ref = 20
f_lower = 18
approximant = IMRPhenomXHM

Each injection has a different distance parameter (50-350 Mpc). 

If you want to make a custom injection, see the injection tutorial.

Edit **inspiral.ini** with the correct injection:

In [ ]:
injection-file = /home/hannah.griggs/nu/banks/gen_plot_injections/autoinjections/pycbcinjs/injectionsingle_150.hdf ## Change "150" to whichever injection you're working on. Make sure this matches your label in run.sh. 

### Good to go! Run the analysis with:

In [ ]:
./run.sh

You will be prompted to enter your password, then it'll be off. 

## Part 2: Reusing matched filtering results from existing analyses

This can be tricky. The overarching steps are:

1. Find the box you want to rerun.
2. Copy over the mmatest-main.map file from that run directory to where you want to store your map files. In there should be the paths to all of the files that could be reused for another run of PyCBC
3. Remove all files post-coinc step. That's a lot of files so you'll need to be careful about this. 

So let's break that down. First, open up the file and get ready to delete a bunch of lines. The lines we are removing represent all the output files from the coincident jobs and onward in the workflow. In other words, everything that depends on running pycbc_coinc_findtrigs. 

Things to keep in mind: 
- There is a logic to this file. It populates in order of job completion, so it's safe to assume that you want to keep everything before the first "COINC" output file shows up.
- This file follows the structure of the main.dax files. If you want to double check anything, open up the dax file in the run directory and trace where the output of the coinc jobs go. 

#### **How to edit your file**
1. Ctrl+F "INSPIRAL"
2. Scroll to the end of each section of INSPIRAL outputs
3. Delete EVERYTHING between INSPIRAL sections and after the last INSPIRAL section EXCEPT 
    - the FIT_OVER_PARAM, FIT_BY_TEMPLATE files.
    - the H1L1-BANK2HDF_SPLITBANK files

O4-specifc Plus:
1. "H1L1-EXCLUDE_ZEROLAG_FULL_DATA_2DET"......hdf
2. "H1L1-FOREGROUND_CENSOR".......xml
3. "H1L1-HDFINJFIND_NSBHSEOBNRV4_INJ_INJECTIONS"........hdf
4. "H1L1-HDFINJFIND_ALL_INJECTIONS-1368975466"........hdf

**Save this edited .map file under a unique name**

Edit runcache.sh to point to the correct .map file:

In [ ]:
  --cache-file maps/inputmap_inj120.map \ ## Path that points to the .map file you want to use 

## Part 3: Using a custom DtDp PDF

Edit analysis_dtdp_noinj.ini to point to the custom PDF you are using:

In [ ]:
statistic-files = file://localhost/home/hannah.griggs/nu/pynu_tests/skyloc/dtdphase/L1H1-stat-O3_c1inj486_0.hdf  https://git.ligo.org/pycbc/offline-analysis/raw/v2.3.1.0/production/o4/kde_files/TEMPLATE_KDE_PYCBCO4_VERSION1.hdf  https://git.ligo.org/pycbc/offline-analysis/raw/v2.3.1.0/production/o4/kde_files/INJECTION_KDE_PYCBCO4_SNR_GT10_VERSION1.hdf ## Change the 1st file here to point to the PDF youre using

Finally, **double check that all your other housekeeping is in order** in runcache.sh (Does the RUNID match your injection and dtdp pdf identifiers? Is your name in INI_LOC and kinit instead of mine?)

### Good to go! Run the analysis with:

In [ ]:
./runcache.sh

You will be prompted to enter your password, then it'll be off. 

## Part 4: Troubleshooting if jobs are struggling:

You'll need to babysit the jobs since they've been having issues with disk space.
Check how the queue is doing from within the run directory with:

In [ ]:
./status

**If a small cluster of jobs fail**, let the analysis get as far as it can until the status updates to (FAILURE).

### Restarting a job that failed

Once it fails, edit the "start" script (in your run output directory) to include the preamble for the run.sh script (for authentication reasons):

In [ ]:
ecp-get-cert --destroy
htdestroytoken
kinit hannah.griggs ## REMEMBER TO CHANGE TO YOUR NAME
unset XDG_RUNTIME_DIR
htgettoken -a vault.ligo.org -i igwn --scopes dqsegdb.read,gwdatafind.read,read:/frames,read:/ligo,read:/virgo,read:/kagra
condor_vault_storer -v igwn
export GWDATAFIND_SERVER="datafind.ligo.org:443"
PEGASUS_PYTHON=/home/ian.harry/conda_envs/pegasus_python/bin/python PATH=/home/ian.harry/conda_envs/pegasus_python/bin/:${PATH}

pegasus-run /local/hannah.griggs/pycbc-tmp_u_1hqa3g/work $@

Then you can restart the job with:

In [ ]:
./start

### Restarting a job that's held

**If jobs are getting held**, see the reason with:

In [ ]:
condor_q better-analyze

This will tell you which job requirements are insufficient and by how much. If memory or disk space are the problem, update held jobs like this:

In [ ]:
condor_qedit -constraint "JOBSTATUS==5" RequestDisk=newrequestamount

Change RequestDisk to RequestMemory as needed, and only request a little over what the jobs seem to need.

Release jobs again with 

In [ ]:
condor_release -constraint "JOBSTATUS==5"